# V4_2の最終的に使うmodel選定のために，どのようにモデルを切り抜いたらいいかを分析するシート
＝現在の方針としては昨年同月でtest,finalともに利益が出ているモデルを切り抜いて，その連続性から安定を判別する．

In [1]:
# import warnings
# warnings.filterwarnings('ignore')

In [30]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from matplotlib import pyplot as plt

import seaborn
from pandas import DataFrame
from tqdm import tqdm
from imblearn.under_sampling import RandomUnderSampler #アンダーサンプリング用
from sklearn.model_selection import train_test_split
import pickle
# 機械学習用
from sklearn.cluster import KMeans #クラスタリング用
from sklearn.ensemble import RandomForestClassifier#ランダムフォレスト
from copy import deepcopy as cp
from sklearn.linear_model import LogisticRegression
import time
import datetime
from dateutil.relativedelta import relativedelta
import os #ディレクトリ作成用
import xgboost as xgb
import sys
from sklearn.preprocessing import StandardScaler#モデルの評価用に標準化する関数
import scipy.stats#モデルの評価用に標準化する関数
import warnings
from pycaret.regression import *#setupが回帰と分類で名前が同じだが機能が違うので，使う方のみインポート（後にインポートした方が使われる)
#from pycaret.classification import *#setupが回帰と分類で名前が同じだが機能が違うので，使う方のみインポート（後にインポートした方が使われる)

#自作のモジュールのインポート
#sys.path.append("..")
import module.master as master
import module.modeling_scores as modeling


In [31]:
#使えそうな関数
def get_season_date(now_date):#日付(datetime型)を渡すと，その日付で購買予測を行う際に使用するデータの区間を返す関数
    use_data_year=now_date.year
    if (now_date.month>=1)and(now_date.month<4):
        use_data_month=1
    elif (now_date.month>=4)and(now_date.month<7):
        use_data_month=4
    elif (now_date.month>=7)and(now_date.month<10):
        use_data_month=7
    elif (now_date.month>=10):
        use_data_month=10
    else:
        print('what???????')
    use_data_date=datetime.datetime(year=use_data_year, month=use_data_month,day=1)
    return use_data_date


def get_4_section_dt(now_date):#今いる区間から直近4区間の開始日をリストで返してくれる関数
    now_sec_date=get_season_date(now_date)
    diff_sec_stdates=[0]*4
    for i in range(len(diff_sec_stdates)):
        diff_sec_stdates[3-i]=now_sec_date- relativedelta(months=3*(i+1))#古い順に日付を入れていく
    return diff_sec_stdates

def get_feature_weights(score_final_df):#相関係数を変数の重みとして取得する関数，（1comあたり1行のdfを返す．）(引数として一つ前の区間,もしくは昨年同月区間のfinalありスコアのdfを渡す)
    plus_m_th=1
    num_hit_th=3
    feature_weights_df=pd.DataFrame()
    for com in sorted(score_final_df['target_com'].value_counts().sort_index().index):
        target_com_df=score_final_df[score_final_df['target_com']==com]
        #ざっと切り捨て，あまりに少ないのは適さないと思うので消す
        target_com_df=target_com_df[target_com_df['plus_month_num_test']>=plus_m_th].copy()#収益がプラスだった月の数で切り捨てる
        target_com_df=target_com_df[target_com_df['num_hit_test']>=num_hit_th].copy()#的中した予測の数で切り捨てる，大体一カ月に一件くらいをイメージ
        target_com_corr=target_com_df.corr()
        target_com_weights=target_com_corr.loc['gain_final']['total_get_test':'diff_mea_med_test']#相関係数を代用した変数ごとの重みづけ        
        feature_weights_df=feature_weights_df.append(target_com_weights)
    feature_weights_df['target_com']=score_final_df['target_com'].value_counts().sort_index().index
    feature_weights_df=feature_weights_df.set_index('target_com')
    feature_weights_df=feature_weights_df.fillna(0)
    return feature_weights_df


def add_std_score(good_model_df,feature_weights):#偏差値と相関係数をもとにしたスコアを与える
    add_score_df=pd.DataFrame()
    for com in sorted(good_model_df['target_com'].value_counts().sort_index().index):
        target_com_df=good_model_df[good_model_df['target_com']==com]
        #display(feature_weights)
        for f_name,weight in feature_weights.items():
            target_com_df.loc[:,'{}_score'.format(f_name)]=scipy.stats.zscore(target_com_df['{}'.format(f_name)].values)
            target_com_df.loc[:,'{}_score'.format(f_name)]=target_com_df['{}_score'.format(f_name)]*weight
        target_com_df=target_com_df.fillna(0)
        target_com_df['score']=0
        for f_name in feature_weights.index:
            target_com_df['score']=target_com_df['score']+target_com_df['{}_score'.format(f_name)]
        #display(target_com_df)
        add_score_df=pd.concat([add_score_df, target_com_df], axis=0)#きめのパラメータで切り落としたので結合
    
    return add_score_df

def add_std_score_DFversion(good_model_df,feature_weights_df):#偏差値と相関係数をもとにしたスコアを与える
    add_score_df=pd.DataFrame()
    for com in sorted(good_model_df['target_com'].value_counts().sort_index().index):
        target_com_df=good_model_df[good_model_df['target_com']==com]
        feature_weights=feature_weights_df.loc[com].copy()
        #display(feature_weights)
        for f_name,weight in feature_weights.items():
            target_com_df.loc[:,'{}_score'.format(f_name)]=scipy.stats.zscore(target_com_df['{}'.format(f_name)].values)
            target_com_df.loc[:,'{}_score'.format(f_name)]=target_com_df['{}_score'.format(f_name)]*weight
        target_com_df=target_com_df.fillna(0)
        target_com_df['score']=0
        for f_name in feature_weights.index:
            target_com_df['score']=target_com_df['score']+target_com_df['{}_score'.format(f_name)]
        #display(target_com_df)
        add_score_df=pd.concat([add_score_df, target_com_df], axis=0)#きめのパラメータで切り落としたので結合
    
    return add_score_df



def select_use_model(model_std_score_df):#相関係数と標準偏差によって与えられたscoreをもとに最終的に使用するモデルを選定する関数
    use_model_df=pd.DataFrame()
    for com in sorted(model_std_score_df['target_com'].value_counts().index):
        target_com_df=model_std_score_df[model_std_score_df['target_com']==com].copy()
        com_selected_row=target_com_df.sort_values('score', ascending=False).iloc[:1]
        #use_model_df=pd.concat([use_model_df, com_selected_row], axis=0)#きめのパラメータで切り落としたので結合
        use_model_df=use_model_df.append(com_selected_row,ignore_index=True)
    return use_model_df


def twin_plt(x,y1,y2,x_n,y1_n,y2_n):#2本軸でのグラフの描写
    fig = plt.figure(figsize=(19,8))
    plt.tick_params(colors='black')
    plt.rcParams["font.size"] = 18
    
    ax1 = fig.add_subplot()
    fig.set_facecolor(color='white')
    ax1.plot(x, y1,label=y1_n, marker="o")
    ax1.tick_params()
    plt.ylim(80, 200)
    ax2 = ax1.twinx()
    #print(y2)
    ax2.plot(x, y2,label=y2_n,color='red', marker="*")
    ax1.axhline(100,ls = "-.",color='green')
    #ax2.tick_params(colors='white')
    ax2.tick_params()

    h1, l1 = ax1.get_legend_handles_labels()
    h2, l2 = ax2.get_legend_handles_labels()
    ax1.legend(h1+h2, l1+l2, loc='lower center')

    #ax1.set_xlabel(x_n,color='white')
    #ax1.set_ylabel(y1_n,color='white')
    #ax2.set_ylabel(y2_n,color='white')
    ax1.set_xlabel(x_n)
    ax1.set_ylabel(y1_n)
    ax2.set_ylabel(y2_n)
    plt.show()
    return None

## 新規関数　

In [32]:
def serch_group(models_df):#モデルシートのパラメータをもとに，パラメータが連続している範囲内で同じ集団とし，集団番号を与える関数
    add_number_df=models_df.copy()
    add_number_df["group_num"]=0
    g_num=0
    g_num_arr=[0]*len(add_number_df)
    diff_index=-1
    i=0
    for index,row in add_number_df.iterrows():
        if index==diff_index+1:
            g_num_arr[i]=g_num
            diff_index=index#現在のインデックスを一つ前として保存
        elif index>diff_index+1:
            g_num=1+g_num
            g_num_arr[i]=g_num
            diff_index=index#現在のインデックスを一つ前として保存
        else:
            print('???????????????????????????????????')
        i+=1
    add_number_df["group_num"]=g_num_arr
    return add_number_df




In [33]:
version='V4_2'#バージョン
#use_modelの書き込み先作成のための箱作り
place_master=master.get_place_master()
place_names=[place_name for place_name in place_master.values()]#会場名のみを収納した配列
for place in place_master.items():
    #print(place[0],place[1],'\n')
    place_name=place[1]
    dir_path = "../bot_database/{place_name}/model_score_{place_name}/use_model/{version}".format(place_name=place_name,version=version)#作成したデータの書き込み先#使用するデータの読み込み
    if os.path.exists(dir_path)==False:
        os.makedirs(dir_path)
    else:
        pass

# 試しにpycharet使う

In [34]:
mode='model_score_period_'
mode_type='period'
score_dfs=list()

#いったんこれ以下でデータの結合を行う
for place_name in place_names[:5]:
    
    concat_df=pd.DataFrame()
    for i in range(4):
        if i<2:
            now_ym=datetime.datetime(year=2020, month=7+(i*3),day=1).strftime('%Y%m%d')
        else:
            now_ym=datetime.datetime(year=2021, month=1+(i*3)-6,day=1).strftime('%Y%m%d')
        print(now_ym)
        dir_path='../bot_database/{place_name}/model_score_{place_name}/v4_score/{version}/{place_name}_{mode}st{now_ym}_finalM3_{version}.csv'.format(place_name=place_name,mode=mode,now_ym=now_ym,version=version)
        model_score_df=pd.read_csv(dir_path, encoding='utf_8_sig')
        model_score_df=model_score_df.drop(["Unnamed: 0"],axis=1)
        concat_df = pd.concat([concat_df, model_score_df])
        concat_df= concat_df.drop(['total_get_final', 'total_use_final','num_com_final','num_pred_final','gain_std_final','num_hit_final','buy_hit_per_final','buy_hit_per_std_final','plus_month_num_final','diff_mea_med_final'], axis=1)
        concat_df=concat_df.fillna(0)
    score_dfs.append(concat_df)

20200701
20201001
20210101
20210401
20200701
20201001
20210101
20210401
20200701
20201001
20210101
20210401
20200701
20201001
20210101
20210401
20200701
20201001
20210101
20210401


## 試しに動かしてみる（サンプルコード代わりにしてちょうだい）

In [35]:
target_scores_df=score_dfs[0]
#exp = setup(target_scores_df,target='gain_final',use_gpu=True,train_size=0.25,data_split_shuffle=False)
exp = setup(target_scores_df,target='gain_final',use_gpu=True,train_size=0.25)

#

,Description,Value
0,session_id,3185
1,Target,gain_final
2,Original Data,"(56448, 16)"
3,Missing Values,False
4,Numeric Features,14
5,Categorical Features,1
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(14112, 14)"


#### アルゴリズムの比較

In [36]:
compare_models( errors="raise")

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,13.1026,662.2532,25.6946,0.7943,0.7356,0.1814,1.2340
et,Extra Trees Regressor,12.5905,675.9533,25.9378,0.7900,0.6910,0.1767,1.8920
xgboost,Extreme Gradient Boosting,16.4249,759.9243,27.5220,0.7640,0.8618,0.2508,0.8690
lightgbm,Light Gradient Boosting Machine,19.4769,918.1158,30.2506,0.7154,0.9741,0.3134,0.0700
dt,Decision Tree Regressor,15.6234,1187.9326,34.4216,0.6293,0.7705,0.2191,0.0890
knn,K Neighbors Regressor,22.0546,1567.7382,39.5239,0.5138,1.0035,0.3216,0.3810
gbr,Gradient Boosting Regressor,30.6191,1890.2731,43.4280,0.4139,1.1996,0.5358,1.5170
lasso,Lasso Regression,41.1221,3062.1143,55.3050,0.0493,1.3874,0.7487,0.0810
br,Bayesian Ridge,41.0451,3062.6782,55.3067,0.0492,1.3836,0.7467,0.0080
lar,Least Angle Regression,41.0278,3064.5012,55.3222,0.0487,1.3827,0.7467,0.0060


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=-1, oob_score=False,
                      random_state=3185, verbose=0, warm_start=False)

 ### 比較したアルゴリズムから良さげなものを作る

In [37]:
et_model=create_model('et')#Extra Trees Regressor

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,12.8681,678.7234,26.0523,0.8117,0.7273,0.1774
1,13.2177,751.2108,27.4082,0.7590,0.6388,0.1898
2,12.6406,834.1688,28.8820,0.7491,0.6395,0.1694
3,12.2662,602.3414,24.5426,0.8200,0.6505,0.1685
4,11.9502,688.4319,26.2380,0.7684,0.7438,0.1613
5,12.8756,718.7302,26.8091,0.7717,0.6886,0.1814
6,11.6245,502.9311,22.4261,0.8230,0.7154,0.1743
7,12.5278,616.1863,24.8231,0.8147,0.7074,0.1969
8,13.3017,762.5098,27.6136,0.7783,0.7020,0.1731
9,12.6328,604.2997,24.5825,0.8046,0.6968,0.1746


In [38]:
print(et_model)

ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                    max_depth=None, max_features='auto', max_leaf_nodes=None,
                    max_samples=None, min_impurity_decrease=0.0,
                    min_impurity_split=None, min_samples_leaf=1,
                    min_samples_split=2, min_weight_fraction_leaf=0.0,
                    n_estimators=100, n_jobs=-1, oob_score=False,
                    random_state=3185, verbose=0, warm_start=False)


In [39]:
rf_model=create_model('rf')#Random Forest Regressor

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,13.3954,636.5490,25.2299,0.8234,0.8131,0.1790
1,13.6460,727.1365,26.9655,0.7667,0.6428,0.1960
2,12.7861,721.3123,26.8573,0.7831,0.7103,0.1756
3,13.0005,681.3298,26.1023,0.7964,0.6977,0.1727
4,12.6613,638.7700,25.2739,0.7851,0.8035,0.1768
5,13.3580,724.8831,26.9237,0.7697,0.7325,0.1813
6,12.2191,495.3492,22.2564,0.8256,0.7562,0.1808
7,13.3566,694.8844,26.3607,0.7910,0.7499,0.1978
8,13.6102,713.7575,26.7162,0.7925,0.6912,0.1805
9,12.9928,588.5601,24.2603,0.8097,0.7588,0.1738


In [40]:
print(rf_model)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=-1, oob_score=False,
                      random_state=3185, verbose=0, warm_start=False)


### 作ったモデルをパラメータチューニング 

In [41]:
tuned_rf_model=tune_model(rf_model)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,23.3416,1235.5628,35.1506,0.6572,1.0520,0.3809
1,22.6067,1073.0362,32.7572,0.6557,0.8446,0.3966
2,22.2832,1213.3961,34.8338,0.6350,0.9224,0.3841
3,23.0854,1270.1240,35.6388,0.6204,0.9013,0.4114
4,23.1888,1151.5846,33.9350,0.6126,1.0324,0.3914
5,24.2725,1361.5859,36.8997,0.5675,0.9760,0.3891
6,22.0183,1041.5590,32.2732,0.6334,1.0063,0.3752
7,23.4534,1325.3932,36.4059,0.6013,0.9849,0.4039
8,23.5847,1271.1171,35.6527,0.6305,0.9196,0.3853
9,23.2133,1151.9789,33.9408,0.6274,0.9474,0.3869


In [42]:
tuned_et_model=tune_model(et_model)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,26.6623,1515.6861,38.9318,0.5795,1.1966,0.4313
1,24.5416,1212.7869,34.8251,0.6109,1.0096,0.4217
2,24.1688,1467.0273,38.3018,0.5588,1.0600,0.4116
3,24.6624,1371.9081,37.0393,0.5900,1.0570,0.4239
4,25.3516,1287.7228,35.8849,0.5668,1.1445,0.4268
5,26.0895,1475.4467,38.4115,0.5313,1.1477,0.4104
6,23.8329,1102.5050,33.2040,0.6119,1.1772,0.4189
7,25.3950,1466.5133,38.2951,0.5589,1.1207,0.4466
8,26.2176,1630.6071,40.3808,0.5260,1.0624,0.4222
9,25.0741,1258.1532,35.4705,0.5931,1.1123,0.4154


## チューニングしたモデルの予測結果を確認する

In [43]:
type(predict_model(tuned_rf_model))

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Random Forest Regressor,23.0783,1256.0007,35.4401,0.6036,0.9615,0.3915


pandas.core.frame.DataFrame

In [44]:
rf_predict_df=predict_model(tuned_rf_model)
rf_predict_df.to_csv('ex_csv/rf_pred.csv')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Random Forest Regressor,23.0783,1256.0007,35.4401,0.6036,0.9615,0.3915


In [45]:
et_predict_df=predict_model(tuned_et_model)

et_predict_df.to_csv('ex_csv/et_pred.csv')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extra Trees Regressor,24.6776,1352.9215,36.7821,0.5730,1.1067,0.4121


## 汎化性能があるか確認する（本当の未知データに対して予測する）

In [54]:
#place_name
#検証用のデータに対して予測する
place_name=place_names[0]
dir_path='../bot_database/{place_name}/model_score_{place_name}/v4_score/{version}/{place_name}_{mode}st20210701_finalM3_{version}.csv'.format(place_name=place_name,mode=mode,version=version)
model_score_df=pd.read_csv(dir_path, encoding='utf_8_sig')
model_score_df=model_score_df.drop(["Unnamed: 0"],axis=1)
model_score_df= model_score_df.drop(['total_get_final', 'total_use_final','num_com_final','num_pred_final','gain_std_final','num_hit_final','buy_hit_per_final','buy_hit_per_std_final','plus_month_num_final','diff_mea_med_final'], axis=1)
model_score_y=model_score_df['gain_final']#本当の収益データ
model_score_x=model_score_df.drop('gain_final',axis=1)


In [55]:
#予測をして，正解と比較する
predict_df = predict_model(tuned_et_model, data=model_score_x)
predict_df['gain_ans']=model_score_y
predict_df.to_csv('ex_csv/pred_ans.csv')

# モデルの選定のために，集団のサイズがどれくらい影響数するのか，また集団サイズの分布について分析する

In [1]:
# place_master=master.get_place_master()
# place_names=[place_name for place_name in place_master.values()]#会場名のみを収納した配列
# #絶対に固定のパラメータたち
# plus_m_th=1

# num_hit_th=3
# version='V4_2'
# mode='model_score_period_'
# mode_type='period'
# #today=datetime.date.today()#今日の日付の取得
# today=datetime.datetime(year=2021, month=10,day=1)
# now_date=today
# for place_name in place_names:
#     #place_name='kiryu'
#     print(place_name,'===============================================================')
#     diff_sec_dates=get_4_section_dt(now_date)#直近４区間の開始日をリストで取得
#     now_date_txt=now_date.strftime('%Y%m%d')
#     diff_sec_dates=diff_sec_dates[1:]#テスト時のみの特別処理！！データが１区間だけ足りないので今回はスルーする
#     #閾値にあてはまった　モデルたちがどんな奴らなのかを確認する
#     for i in range(len(diff_sec_dates)):
#         #i=0
#         sec_date=diff_sec_dates[i]
#         sec_date_txt=sec_date.strftime('%Y%m%d')

#         #昨年同月のfinalあり分析結果のシートから使えそうなcom，集団を作成し，パラメータが参照できるように切り抜く
#         date_cal_last=sec_date- relativedelta(months=12)#昨年同月の区間の日付を切り抜く
#         date_cal_last_txt=date_cal_last.strftime('%Y%m%d')
#         dir_path='../bot_database/{place_name}/model_score_{place_name}/v4_score/{version}/{place_name}_{mode}st{now_ym}_finalM3_{version}.csv'.format(place_name=place_name,mode=mode,now_ym=date_cal_last_txt,version=version)
#         model_score_df=pd.read_csv(dir_path, encoding='utf_8_sig')
#         model_score_df=model_score_df.drop(["Unnamed: 0"],axis=1)
#         #並べる順番を変える，閾値をはじめの方に持ってきた
#         model_score_df=model_score_df.sort_values(['target_com', 'threshold','depth','target_per'])
#         model_score_df = model_score_df.reset_index(drop=True)
#         #print('before_drop[num_com]:',len(model_score_df))#収益で切り落とす前のモデル数
#         both_plus_models=model_score_df[(model_score_df['gain_test']>100)&(model_score_df['gain_final']>100)].copy()#まずは昨年同月の，testとfinalともに利益が出ていたものを切り抜く
#         #print('after_drop[num_com]:',len(both_plus_models))#収益で切り落とした後のモデル数
#         models_has_g_num_df=serch_group(both_plus_models)#モデルのパラメータから集団番号を付与
        
        
#         #試しに決めの値で集団長を指定して切り抜く
#         len_th=7
#         g_num_len=models_has_g_num_df['group_num'].value_counts()#集団番号ごとの長さ
#         numbers_g_s=g_num_len.loc[g_num_len>=len_th]#集団の長さが閾値以上のものをvalue_counts()の結果から切り抜く
#         group_df=pd.DataFrame()
#         for g_num in numbers_g_s.index:
#             target_g_num_df=models_has_g_num_df[models_has_g_num_df['group_num']==g_num]
#             group_df=pd.concat([group_df,target_g_num_df])
#         g_nums_counts=group_df['group_num'].value_counts(sort=False)
#         #集団の真ん中のモデルを切り抜く
#         center_models_df=pd.DataFrame(columns=group_df.columns)#真ん中で切り抜いたモデルのか格納先
#         for index,len_group in g_nums_counts.items():
#             model_groups_df=group_df[group_df['group_num']==index].copy()
#             target_row=model_groups_df.iloc[int(len_group)-1]
#             center_models_df=center_models_df.append(target_row)

#         #絶対的な的中数でさらにモデルを切り抜く
#         hit_th=5
#         good_models_df=center_models_df[(center_models_df['num_hit_test']>hit_th) & (center_models_df['num_hit_final']>hit_th)].copy()
#         #good_models_df=center_models_df[(center_models_df['num_hit_test']>2) & (center_models_df['num_hit_final']>2)].copy()
#         selected_models_df=pd.DataFrame(columns=group_df.columns)#真ん中で切り抜いたモデルのか格納先
#         for com in good_models_df['target_com'].value_counts().index:
#             good_models_com_df=good_models_df[good_models_df['target_com']==com].copy()
#             good_models_com_row=good_models_com_df.sort_values(['num_hit_final','gain_final'],ascending=[False, False]).iloc[0]#的中数，finalゲインでsp－として一番大きいものにする
#             selected_models_df=selected_models_df.append(good_models_com_row)
            
            
#         #今年のスコアシートと，昨年同月から選んだモデルたちを渡して，今年で使うモデルを切り抜く
#         dir_path='../bot_database/{place_name}/model_score_{place_name}/v4_score/{version}/{place_name}_{mode}st{now_ym}_finalM3_{version}.csv'.format(place_name=place_name,mode=mode,now_ym=sec_date_txt,version=version)
#         now_model_score_df=pd.read_csv(dir_path, encoding='utf_8_sig')#今年のスコアシートの読み込み
#         now_model_score_df=now_model_score_df.drop(["Unnamed: 0"],axis=1)
#         use_models_df=get_same_para_model(selected_models_df,now_model_score_df)
#         display(use_models_df)

#         #display(use_models_df['gain_final'])
    
    
    